In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
import os
import sys
import shutil
import numpy as np
from itertools import cycle,islice
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset, IterableDataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from itertools import combinations
from tqdm import tqdm
import json
os.chdir("/content/drive/MyDrive/project")
from entity import person
from paths import *
from retabulate import tabulate,init_history,mean_history
import matplotlib.pyplot as plt
import pickle as pkl

In [ ]:
os.makedirs("/content/drive/MyDrive/project/modelnew/")

In [ ]:
N_LATENTS=512
CUDA=torch.cuda.is_available()
EPOCHS=500 
OUT_DIR="/content/drive/MyDrive/project/modelnew/"
lambda_image=1
lambda_speech=100
learning_rate= 1e-3
annealing_epochs=200
log_interval=100
batch_size= 128
N_mini_batches=48000//batch_size
N_mini_batches_val=188
N_mini_batches_test=317

In [ ]:
p = person("Farsi", 'Kannada',dtype=np.float32)

In [ ]:
class MyIterableDataset(IterableDataset):
    def __init__(self,p, mode):
        self.data=p
        self.mode=mode
    def parse_data(self):
        if self.mode=="train":
            for sample in self.data._sample(self.data.mnist_1_X_train, 
                                            self.data.mnist_2_X_train, 
                                            self.data.speech_X_train, 
                                            self.data.mnist_1_y_train, 
                                            self.data.mnist_2_y_train, 
                                            self.data.speech_y_train):
                yield sample
        elif self.mode=="val":
            for sample in self.data._sample(self.data.mnist_1_X_valid, 
                                            self.data.mnist_2_X_valid, 
                                            self.data.speech_X_valid, 
                                            self.data.mnist_1_y_valid, 
                                            self.data.mnist_2_y_valid, 
                                            self.data.speech_y_valid):
                yield sample
        else:
            for sample in self.data._sample(self.data.mnist_1_X_test, 
                                            self.data.mnist_2_X_test, 
                                            self.data.speech_X_test, 
                                            self.data.mnist_1_y_test, 
                                            self.data.mnist_2_y_test, 
                                            self.data.speech_y_test):
                yield sample
    # def get_stream(self):
    #     return cycle(self.parse_data())
    def __iter__(self):
        return self.parse_data()

In [ ]:
iterable_dataset=MyIterableDataset(p, mode="train")
train_loader=DataLoader(iterable_dataset,batch_size=batch_size)
iterable_dataset_val=MyIterableDataset(p, mode="val")
val_loader=DataLoader(iterable_dataset_val,1)
iterable_dataset_test=MyIterableDataset(p, mode="test")
test_loader=DataLoader(iterable_dataset_test,1)

In [ ]:

for batch_idx,((mnist1, mnist2, speech), label_y) in enumerate(test_loader):
    # print(mnist2)
    break
# print(batch_idx)

# Model

In [ ]:
class MVAE(nn.Module):
    """Multimodal Variational Autoencoder.
    @param n_latents: integer
                      number of latent dimensions
    """
    def __init__(self, n_latents):
        super(MVAE, self).__init__()
        self.image1_encoder = ImageEncoder(n_latents)
        # self.image1_encoder=self.image1_encoder.apply(self.weights_init)
        self.image1_decoder = ImageDecoder(n_latents)
        self.image2_encoder = ImageEncoder(n_latents)
        # self.image2_encoder=self.image2_encoder.apply(self.weights_init)
        self.image2_decoder = ImageDecoder(n_latents)
        self.speech_encoder  = SpeechEncoder(n_latents)
        # self.speech_encoder=self.speech_encoder.apply(self.weights_init)
        self.speech_decoder  = SpeechDecoder(n_latents)
        self.label_decoder = LabelDecoder(n_latents)

        self.experts       = ProductOfExperts()
        self.n_latents     = n_latents
    def weights_init(self,m):
        if isinstance(m, nn.Linear):
            torch.nn.init.zeros_(m.weight)
            torch.nn.init.zeros_(m.bias)
    
    def reparametrize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
          return mu

    def forward(self, image1=None,image2=None, speech=None):
        mu, logvar = self.infer(image1,image2, speech)
        # reparametrization trick to sample
        z          = self.reparametrize(mu, logvar)
        # reconstruct inputs based on that gaussian
        img1_recon  = self.image1_decoder(z)
        img2_recon  = self.image2_decoder(z)
        sp_recon  = self.speech_decoder(z)
        label_recon=self.label_decoder(z)
        return img1_recon,img2_recon, sp_recon,label_recon, mu, logvar

    def infer(self, image1=None,image2=None, speech=None):

        batch_size = image1.size(0) if image1 is not None else image2.size(0) if image2 is not None else speech.size(0)
        use_cuda   = next(self.parameters()).is_cuda  # check if CUDA
        # initialize the universal prior expert
        mu, logvar = prior_expert((1, batch_size, self.n_latents), 
                                  use_cuda=use_cuda)
        if image1 is not None:
            img1_mu, img1_logvar = self.image1_encoder(image1)
            mu     = torch.cat((mu, img1_mu.unsqueeze(0)), dim=0)
            logvar = torch.cat((logvar, img1_logvar.unsqueeze(0)), dim=0)
        if image2 is not None:
            img2_mu, img2_logvar = self.image2_encoder(image2)
            mu     = torch.cat((mu, img2_mu.unsqueeze(0)), dim=0)
            logvar = torch.cat((logvar, img2_logvar.unsqueeze(0)), dim=0)

        if speech is not None:
            sp_mu, sp_logvar = self.speech_encoder(speech)
            mu     = torch.cat((mu, sp_mu.unsqueeze(0)), dim=0)
            logvar = torch.cat((logvar, sp_logvar.unsqueeze(0)), dim=0)

        # product of experts to combine gaussians
        mu, logvar = self.experts(mu, logvar)
        return mu, logvar

In [ ]:
class ImageEncoder(nn.Module):
    """Parametrizes q(z|x).
    @param n_latents: integer
                      number of latent dimensions
    """
    def __init__(self, n_latents):
        super(ImageEncoder, self).__init__()
        self.fc1   = nn.Linear(784, 2048)
        self.fc1_2   = nn.Linear(2048, 1024)
        self.fc2   = nn.Linear(1024, 512)
        self.fc31  = nn.Linear(512, n_latents)
        self.fc32  = nn.Linear(512, n_latents).apply(self.weights_init)
        self.swish = Swish()
    def weights_init(self,m):
        torch.nn.init.zeros_(m.weight)
        torch.nn.init.zeros_(m.bias)
    def forward(self, x):
        xx=x.view(-1, 784)
        h= self.fc1(xx.float())
        h = self.swish(h)
        h = self.swish(self.fc1_2(h))
        h = self.swish(self.fc2(h))
        return self.fc31(h), self.fc32(h)

In [ ]:
class ImageDecoder(nn.Module):
    """Parametrizes p(x|z).
    @param n_latents: integer
                      number of latent dimensions
    """
    def __init__(self, n_latents):
        super(ImageDecoder, self).__init__()
        self.fc1   = nn.Linear(n_latents, 512)
        self.fc2   = nn.Linear(512, 512)
        self.fc3   = nn.Linear(512, 512)
        self.fc3_4   = nn.Linear(512, 512)
        self.fc4   = nn.Linear(512, 784)
        self.swish = Swish()

    def forward(self, z):
        h = self.swish(self.fc1(z))
        h = self.swish(self.fc2(h))
        h = self.swish(self.fc3(h))
        h = self.swish(self.fc3_4(h))
        return self.fc4(h)

In [ ]:
class SpeechEncoder(nn.Module):
    """Parametrizes q(z|y).
    @param n_latents: integer
                      number of latent dimensions
    """
    def __init__(self, n_latents):
        super(SpeechEncoder, self).__init__()
        self.fc1   = nn.Linear(13, 512)
        self.fc2   = nn.Linear(512, 512)
        self.fc31  = nn.Linear(512, n_latents)
        self.fc32  = nn.Linear(512, n_latents).apply(self.weights_init)
        self.swish = Swish()
    def weights_init(self,m):
        torch.nn.init.zeros_(m.weight)
        torch.nn.init.zeros_(m.bias)
    def forward(self, x):
        h = self.swish(self.fc1(x.float()))
        h = self.swish(self.fc2(h))
        return self.fc31(h), self.fc32(h)

In [ ]:
#TODO
class SpeechDecoder(nn.Module):
    """Parametrizes p(y|z).
    @param n_latents: integer
                      number of latent dimensions
    """
    def __init__(self, n_latents):
        super(SpeechDecoder, self).__init__()
        self.fc1   = nn.Linear(n_latents, 512)
        self.fc2   = nn.Linear(512, 512)
        self.fc3   = nn.Linear(512, 512)
        self.fc4   = nn.Linear(512, 13)
        self.swish = Swish()

    def forward(self, z):
        h = self.swish(self.fc1(z))
        h = self.swish(self.fc2(h))
        h = self.swish(self.fc3(h))
        return self.fc4(h)  # NOTE: no softmax here. See train.py

In [ ]:
class LabelDecoder(nn.Module):
    """Parametrizes p(y|z).
    @param n_latents: integer
                      number of latent dimensions
    """
    def __init__(self, n_latents):
        super(LabelDecoder, self).__init__()
        self.fc1   = nn.Linear(n_latents, 512)
        self.fc2   = nn.Linear(512, 512)
        self.fc3   = nn.Linear(512, 512)
        self.fc4   = nn.Linear(512, 10)
        self.swish = Swish()

    def forward(self, z):
        h = self.swish(self.fc1(z))
        h = self.swish(self.fc2(h))
        h = self.swish(self.fc3(h))
        return self.fc4(h)

In [ ]:
class ProductOfExperts(nn.Module):
    """Return parameters for product of independent experts.
    See https://arxiv.org/pdf/1410.7827.pdf for equations.
    @param mu: M x D for M experts
    @param logvar: M x D for M experts
    """
    def forward(self, mu, logvar, eps=1e-8):
        var       = torch.exp(logvar) + eps
        # precision of i-th Gaussian expert at point x
        T         = 1. / (var + eps)
        pd_mu     = torch.sum(mu * T, dim=0) / torch.sum(T, dim=0)
        pd_var    = 1. / torch.sum(T, dim=0)
        pd_logvar = torch.log(pd_var + eps)
        return pd_mu, pd_logvar

In [ ]:
class Swish(nn.Module):
    """https://arxiv.org/abs/1710.05941"""
    def forward(self, x):
        # return x * F.sigmoid(x)
        return x *( torch.tanh(F.softplus(x)))

In [ ]:
class Mish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        #inlining this saves 1 second per epoch (V100 GPU) vs having a temp x and then returning x(!)
        return x *( torch.tanh(F.softplus(x)))

In [ ]:
def prior_expert(size, use_cuda=False):
    """Universal prior expert. Here we use a spherical
    Gaussian: N(0, 1).
    @param size: integer
                 dimensionality of Gaussian
    @param use_cuda: boolean [default: False]
                     cast CUDA on variables
    """
    mu     = Variable(torch.zeros(size))
    logvar = Variable(torch.zeros(size))
    if use_cuda:
        mu, logvar = mu.cuda(), logvar.cuda()
    return mu, logvar

# Training

In [ ]:
def elbo_loss(recon_image1=None, image1=None,recon_image2=None, image2=None, recon_speech=None, speech=None,recon_label=None,y=None, mu=None, logvar=None,
              lambda_image=1.0, lambda_speech=1.0, annealing_factor=1):
    """Compute the ELBO for an arbitrary number of data modalities.
    @param recon: list of torch.Tensors/Variables
                  Contains one for each modality.
    @param data: list of torch.Tensors/Variables
                 Size much agree with recon.
    @param mu: Torch.Tensor
               Mean of the variational distribution.
    @param logvar: Torch.Tensor
                   Log variance for variational distribution.
    @param lambda_image: float [default: 1.0]
                         weight for image BCE
    @param lambda_attr: float [default: 1.0]
                        weight for attribute BCE
    @param annealing_factor: float [default: 1]
                             Beta - how much to weight the KL regularizer.
    """
    # assert len(recon) == len(data), "must supply ground truth for every modality."
    # n_modalities = len(recon)
    batch_size   = mu.size(0)


    image1_bce,image2_bce,speech_mse,label_ce  = 0,0,0,0  # reconstruction cost
    if recon_image1 is not None and image1 is not None:
        # image1_bce = torch.mean(torch.sum(binary_cross_entropy_with_logits(
        #     recon_image1.view(-1, 1 * 28 * 28), 
        #     image1.view(-1, 1 * 28 * 28)), dim=1))
        image1_bce= torch.mean(torch.sum(nn.BCEWithLogitsLoss(reduction='none')(
            recon_image1.view(-1, 1 * 28 * 28), 
            image1.view(-1, 1 * 28 * 28)),dim=1),dim=0)
        # image1_bce= torch.mean(torch.sum(nn.MSELoss(reduction='none')(
        #     recon_image1.view(-1, 1 * 28 * 28), 
        #     image1.view(-1, 1 * 28 * 28)),dim=1),dim=0)
    # print("IMAGE1 RECONSTRUCTION",image1_bce)
    if recon_image2 is not None and image2 is not None:
        # image2_bce = torch.mean(torch.sum(binary_cross_entropy_with_logits(
        #     recon_image2.view(-1, 1 * 28 * 28), 
        #     image2.view(-1, 1 * 28 * 28)), dim=1))
        image2_bce= torch.mean(torch.sum(nn.BCEWithLogitsLoss(reduction='none')(
            recon_image2.view(-1, 1 * 28 * 28), 
            image2.view(-1, 1 * 28 * 28)),dim=1),dim=0)
        # image2_bce= torch.mean(torch.sum(nn.MSELoss(reduction='none')(
        #     recon_image2.view(-1, 1 * 28 * 28), 
        #     image2.view(-1, 1 * 28 * 28)),dim=1),dim=0)
    # print("IMAGE2 RECONSTRUCTION",image2_bce)    
    if recon_speech is not None and speech is not None:  # this is for an attribute
        # print(recon_speech,speech, type(recon_speech),type(speech))
        loss= nn.MSELoss(reduction='none')
        speech_mse = torch.mean(torch.sum(loss(recon_speech, speech),dim=1),dim=0)
    # print("SPEECH RECONSTRUCTION",speech_mse)
    if recon_label is not None and y is not None:
        label_ce=torch.mean(nn.CrossEntropyLoss(reduction='none')(recon_label,torch.argmax(y,dim=1)),dim=0)
    # print("CLASSIFICATION",label_ce)
    KLD = -0.5 * torch.mean(torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1),dim=0)
    # print("KL DIVERGENCE",KLD)
    # KLD=0
    ELBO = lambda_image * image1_bce + lambda_image * image2_bce + lambda_speech * speech_mse+label_ce + annealing_factor * KLD
    return ELBO,image1_bce,image2_bce,speech_mse,label_ce

In [ ]:
def binary_cross_entropy_with_logits(input, target):
    """Sigmoid Activation + Binary Cross Entropy
    @param input: torch.Tensor (size N)
    @param target: torch.Tensor (size N)
    @return loss: torch.Tensor (size N)
    """
    if not (target.size() == input.size()):
        raise ValueError("Target size ({}) must be the same as input size ({})".format(
            target.size(), input.size()))

    return (torch.clamp(input, 0) - input * target 
            + torch.log(1 + torch.exp(-torch.abs(input))))

In [ ]:
def cross_entropy(input, target, eps=1e-6):
    """k-Class Cross Entropy (Log Softmax + Log Loss)
    
    @param input: torch.Tensor (size N x K)
    @param target: torch.Tensor (size N x K)
    @param eps: error to add (default: 1e-6)
    @return loss: torch.Tensor (size N)
    """
    if not (target.size(0) == input.size(0)):
        raise ValueError(
            "Target size ({}) must be the same as input size ({})".format(
                target.size(0), input.size(0)))

    log_input = F.log_softmax(input + eps, dim=1)
    y_onehot = Variable(log_input.data.new(log_input.size()).zero_())
    y_onehot = y_onehot.int().scatter_(1, target.unsqueeze(1), 1)
    loss = y_onehot * log_input
    return -loss

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def save_checkpoint(state, is_best, folder='./', filename='checkpoint.pth.tar'):
    if not os.path.isdir(folder):
        os.mkdir(folder)
    torch.save(state, os.path.join(folder, filename))
    if is_best:
        shutil.copyfile(os.path.join(folder, filename),
                        os.path.join(folder, 'model_best.pth.tar'))


def load_checkpoint(file_path, use_cuda=False):
    checkpoint = torch.load(file_path) if use_cuda else \
        torch.load(file_path, map_location=lambda storage, location: storage)
    model = MVAE(checkpoint['n_latents'])
    model.load_state_dict(checkpoint['state_dict'])
    return model

In [ ]:

# if not os.path.isdir('./trained_models'):
#     os.makedirs('./trained_models')

# crop the input image to 64 x 64
preprocess_data = transforms.Compose([transforms.Resize(64),
                                        transforms.CenterCrop(64),
                                        transforms.ToTensor()])

model     = MVAE(N_LATENTS)
optimizer = optim.Adam(model.parameters(), lr=learning_rate,amsgrad=True)
# optimizer=optim.RMSprop(model.parameters(),lr=learning_rate)
if CUDA:
    model.cuda()

In [ ]:
def train(epoch,History):
    model.train()
    train_loss_meter = AverageMeter()
    history=init_history()
    for batch_idx, ((image1,image2,speech),y) in enumerate(train_loader):
        if epoch < annealing_epochs:
            # compute the KL annealing factor for the current mini-batch in the current epoch
            annealing_factor = (float(batch_idx + (epoch - 1) * N_mini_batches + 1) /
                                float(annealing_epochs * N_mini_batches))
        else:
            # by default the KL annealing factor is unity
            annealing_factor = 1.0

        if CUDA:
            image1     = image1.cuda().float()
            image2     = image2.cuda().float()
            speech     = speech.cuda().float()
            y     = y.cuda()
        image1         = Variable(image1)
        image2         = Variable(image2)
        speech         = Variable(speech)
        y         = Variable(y)
          # convert tensor to list
        batch_size    = len(image1)

        # refresh the optimizer
        optimizer.zero_grad()

        # train_loss    = 0  # accumulate train loss here so we don't store a lot of things.
        history,train_loss= tabulate(model,history,image1,image2,speech,y,batch_idx,annealing_factor,lambda_image,lambda_speech)
        # assert n_elbo_terms == (len(attrs) + 1) + 1 + args.approx_m  # N + 1 + M
        train_loss_meter.update(train_loss.data, len(image1))
        
        # compute and take gradient step
        train_loss.float().backward()
        optimizer.step()

        

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAnnealing-Factor: {:.3f}'.format(
                epoch, batch_idx * batch_size, N_mini_batches*batch_size,
                100. * batch_idx / N_mini_batches, train_loss_meter.avg, annealing_factor))
    
    
    
    History=mean_history(history,History)



    print('====> Epoch: {}\tLoss: {:.4f}'.format(epoch, train_loss_meter.avg))
    return History
def val(epoch,History):
    model.eval()
    history=init_history()
    for batch_idx, ((image1,image2,speech),y) in enumerate(val_loader):
        # count+=1
        if epoch < annealing_epochs:
            # compute the KL annealing factor for the current mini-batch in the current epoch
            annealing_factor = (float(batch_idx + (epoch - 1) * N_mini_batches_val + 1) /
                                float(annealing_epochs * N_mini_batches_val))
        else:
            # by default the KL annealing factor is unity
            annealing_factor = 1.0
        if CUDA:
            image1     = image1.cuda()
            image2     = image2.cuda()
            speech     = speech.cuda()
            y     = y.cuda()
        image1         = Variable(image1)
        image2         = Variable(image2)
        speech         = Variable(speech)
        y         = Variable(y)
        batch_size = image1.size(0)
        
        history,valid_loss= tabulate(model,history,image1,image2,speech,y,batch_idx,annealing_factor,lambda_image,lambda_speech)
        # val_loss+=history['m1m2m3']['elbo']
        
        # pbar.update()
    History=mean_history(history,History)
    # pbar.close()
    val_loss=History['m1m2m3']['elbo'][-1]
    print('====> Test Loss: {:.4f}'.format(val_loss))
    return val_loss,History

In [ ]:
def val1(epoch):
    model.eval()
    val_loss = 0
    count=0
    # for simplicitly, here i'm only going to track the joint loss. 
    # pbar = tqdm(total=len(test_loader))
    for batch_idx, ((image1,image2,speech),y) in enumerate(val_loader):
        count+=1
        if epoch < annealing_epochs:
            # compute the KL annealing factor for the current mini-batch in the current epoch
            annealing_factor = (float(batch_idx + (epoch - 1) * N_mini_batches_val + 1) /
                                float(annealing_epochs * N_mini_batches_val))
        else:
            # by default the KL annealing factor is unity
            annealing_factor = 1.0
        if CUDA:
            image1     = image1.cuda().float()
            image2     = image2.cuda().float()
            speech     = speech.cuda().float()
            y     = y.cuda().float()
        image1         = Variable(image1)
        image2         = Variable(image2)
        speech         = Variable(speech)
        y         = Variable(y)
        batch_size = image1.size(0)
        
        recon_image1, recon_image2, recon_speech, recon_label, mu, logvar = model(image1,image2,speech)
        elbo,image1_bce,image2_bce,speech_mse,label_ce= elbo_loss(recon_image1,image1,recon_image2, image2,recon_speech, speech,recon_label,y , mu, logvar, 
                            lambda_image=lambda_image, lambda_speech=lambda_speech,
                            annealing_factor=annealing_factor)
        val_loss+=elbo
        # pbar.update()

    # pbar.close()
    val_loss /= count
    print('====> Test Loss: {:.4f}'.format(val_loss))
    return val_loss

In [ ]:
best_loss = 100000
best_val_loss=1000000
count=0
History=init_history()
# History_val=init_history()
best_epoch=0
val_losses=[]
for epoch in range(1, EPOCHS + 1):
    History=train(epoch,History)
    print(History)
    # val_loss,History_val=val(epoch,History_val)
    # loss = test(epoch)
    val_loss=val1(epoch)
    # val_losses.append(val_loss)
    is_best = val_loss < best_loss
    if is_best:
        best_epoch=epoch
    if is_best:
        save_checkpoint({
        'state_dict': model.state_dict(),
        'best_loss': best_loss,
        'n_latents': N_LATENTS,
        'optimizer' : optimizer.state_dict(),
    }, is_best, folder=OUT_DIR)
    if (val_loss>best_loss) or torch.abs(val_loss-best_loss)<1e-1:
        # print(val_loss,best_loss)
        count+=1
        if count==4:
            History["best_epoch"]=best_epoch
            History["best_loss"]=best_loss
            History["val_loss"]=val_losses
            with open(OUT_DIR+'/train_history.pkl','wb') as fout:
                pkl.dump(History, fout, protocol=pkl.HIGHEST_PROTOCOL)
            # with open(OUT_DIR+'/val_history.json','rb') as f:
            #     json.dumps(History_val,f)
            break
    else:
        count=0
    best_loss = min(val_loss, best_loss)
    # save the best model and current model
    
    

In [ ]:
with open(OUT_DIR+'/train_history.pkl','rb') as f:
    History=pkl.load(f)

In [ ]:
for keys in History.keys():
    print(best_epoch)
    print(keys)
    print(History[keys]['label_acc'][best_epoch-1])
    # print(History[keys]['image2_bce'][0])
# print(History['total_loss'])

In [ ]:
print(History['m1'].keys())

In [ ]:
#added manually due to cuda issues
val_losses=[359.17,282.2,256.1,241.4,229.1,222.5,215.88,211.67,209,206.6,204.1,202,201.2,199.75,198.72,198.68,198.27,197.4, 196.4,195.34,196.36,195.1,194.58,194.93,193.89,193.87,193.98,193.51,194.35,193.75,193.74,193.42]

In [ ]:
import matplotlib.pyplot as plt
x=np.arange(len(val_losses))
loss=History['m1m2m3']['elbo'][:]
acc=History['m3']['label_acc'][:]
fig, ax1 = plt.subplots()
ax1.set_xlabel('Epochs')
ax1.set_ylabel('ELBO')
# ax1.plot(t, data1, color=color)
ax1.tick_params(axis='y')
ax1.plot(x,loss)
ax1.plot(x,val_losses)
# plt.plot(x,acc)
# plt.xlabel('Epochs')
# plt.ylabel('ELBO')
plt.title('Figure 4: Loss(ELBO) and label accuracy convergence')
ax1.legend(['Training Loss','Validation Loss'])
# 

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:red'
ax2.set_ylabel('Accuracy')  # we already handled the x-label with ax1
ax2.plot(x, acc, color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.legend(['Accuracy'],loc='center right')
fig.tight_layout()
# plt.annotate('learning rate=1e-3', xy=(10, 1800), xytext=(15, 1600))
plt.savefig(OUT_DIR+'/final_loss.png')
plt.show()



In [ ]:
print(len(History['m1m2m3']['label_acc'][:]))

In [ ]:
model=load_checkpoint(OUT_DIR+'/checkpoint.pth.tar',use_cuda=True)
device = torch.device("cuda")
model.to(device)

In [ ]:
model.eval()
val_loss = 0
count=0
# for simplicitly, here i'm only going to track the joint loss. 
# pbar = tqdm(total=len(test_loader))
for _ in range(10):
    for batch_idx, ((image1,image2,speech),y) in enumerate(val_loader):
        imorig1=image1
        imorig2=image2
        # print(torch.argmax(y))
        label=np.argmax(y).cpu().detach().numpy()
        
        if label==1:
            print(label)
            break
    
    count+=1
    if CUDA:
        image1     = image1.cuda().float()
        image2     = image2.cuda().float()
        speech     = speech.cuda().float()
        y     = y.cuda().float()
    image1         = Variable(image1)
    image2         = Variable(image2)
    speech         = Variable(speech)
    y         = Variable(y)
    batch_size = image1.size(0)

    recon_image1, recon_image2, recon_speech, recon_label, mu, logvar = model(speech=speech)
    # print(recon_image1.size())
    recon_image1=recon_image1.view(28,28)
    recon_image2=recon_image2.view(28,28)
    imorig1=imorig1.view(28,28)
    imorig2=imorig2.view(28,28)
    im1=recon_image1.cpu().detach().numpy()
    im2=recon_image2.cpu().detach().numpy()
    imorig1=imorig1.cpu().detach().numpy()
    imorig2=imorig2.cpu().detach().numpy()
    # if not os.path.isdir(OUT_DIR+"/val_m2/"):
    #     os.makedirs(OUT_DIR+"/val_m2/")
    plt.imshow(im1)
    plt.savefig(OUT_DIR+"/val_m3/im1recon_"+str(label)+".png")
    plt.show()
    plt.imshow(imorig1)
    plt.savefig(OUT_DIR+"/val_m3/im1orig_"+str(label)+".png")
    plt.show()
    plt.imshow(im2)
    plt.savefig(OUT_DIR+"/val_m3/im2recon_"+str(label)+".png")
    plt.show()
    plt.imshow(imorig2)
    plt.savefig(OUT_DIR+"/val_m3/im2orig_"+str(label)+".png")
    plt.show()
    if label==1:
        print(label)
        break

In [ ]:
z= model.reparametrize(mu,logvar)
print(z.size())

In [ ]:
from copy import deepcopy
def discover_disentanglement(idx, change, num=100):
  z= model.reparametrize(mu,logvar).clone().detach()
  Z = torch.randn(num,512).to(device)
  sub = [-change*i for i in range(1,num//2+1)][::-1]
  add = [change*i for i in range(1,num//2+1)]
  print(sub+add)
  for i, ch in enumerate(sub+add):
    Z[i] = deepcopy(z)
    Z[i][idx] += ch 
  return Z
tens=discover_disentanglement(25,5)
 

In [ ]:
tens

In [ ]:
image1_recons=model.image1_decoder(tens)

In [ ]:
image1_recons=image1_recons.view(-1,28,28)
image1_recons.size()

In [ ]:
import torchvision
from PIL import Image
im = image1_recons[0,:,:].cpu().detach().numpy()
plt.imshow(im)
plt.show()

In [ ]:
w=10
h=10
fig=plt.figure(figsize=(10, 10))
columns = 10
rows = 10
for i in range(0, columns*rows):
    img = image1_recons[i,:,:].cpu().detach().numpy()
    fig.add_subplot(rows, columns, i+1)
    plt.axis('off')
    # fig.set_aspect('equal')
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.imshow(img,interpolation='nearest')
# plt.show()
plt.savefig(OUT_DIR+"/temp.png",bbox_inches='tight', pad_inches = 0)